## **Accidents de la route en Île-de-France sur l'année 2022**

**Objectifs**: L'objectif de ce notebook est de produire une carte renvoyant les 25 localisations avec le plus d'accidents de la route en 2022. L'idée étant d'avertir l'utilisateur lors de son trajet s'il venait à emprunter l'un de ces tronçons.

In [2]:
import pandas as pd
import os
import folium as fo

In [ ]:
cwd = os.getcwd()  ## Obtient le chemin absolu du répertoire de travail actuel

# Chemin complet du fichier Excel "accidents_2022_idf.xlsx" dans le répertoire de travail
accidents_2022idf_path = os.path.join(cwd, "accidents_2022_idf.xlsx")

# Charge les données du fichier Excel dans un DataFrame nommé accidents_2022idf_carac
accidents_2022idf_carac = pd.read_excel(accidents_2022idf_path)

# Remplace les virgules par des points et convertit la colonne "Latitude" en flottants
accidents_2022idf_carac["Latitude"] = accidents_2022idf_carac["Latitude"].str.replace(',','.').astype(float)

# Remplace les virgules par des points et convertit la colonne "Longitude" en flottants
accidents_2022idf_carac["Longitude"] = accidents_2022idf_carac["Longitude"].str.replace(',','.').astype(float)


In [7]:
#on trie les données d'accident en fonction de leur localisation
dict_accidents_2022idf = accidents_2022idf_carac.groupby(["Commune","Adresse"]).groups
#on trie le dictionnaire obtenu
dict_trie_accidents_2022idf=dict(sorted(dict_accidents_2022idf.items(),key=lambda item : len(item[1]),reverse=True))


In [11]:
dict_trie_accidents_2022idf

{('93066 - Saint-Denis',
  'AUTOROUTE A1'): Int64Index([  43,   51,  126,  377,  539,  558,  606,  634,  657,  918, 1020,
             1125, 1389, 1542, 1838, 2017, 2052, 2074, 2149, 2200, 2393, 2773,
             2794, 2884],
            dtype='int64'),
 ('75108 - Paris 8e Arrondissement',
  'AVENUE DES CHAMPS ELYSEES'): Int64Index([108, 1050, 1440, 1806, 1966, 1976, 2060, 2312, 2917, 2918, 2923,
             2968],
            dtype='int64'),
 ('93027 - Courneuve',
  'AUTOROUTE A86'): Int64Index([110, 714, 1021, 1164, 1397, 1424, 1783, 1929, 1980, 1981, 2001,
             2075],
            dtype='int64'),
 ('92036 - Gennevilliers',
  'AUTOROUTE A15'): Int64Index([122, 238, 314, 317, 340, 716, 1101, 1167, 1333, 1631, 2953], dtype='int64'),
 ('94034 - Fresnes',
  'AUTOROUTE A86'): Int64Index([103, 232, 519, 559, 564, 1723, 1730, 2586, 2625, 2707, 2938], dtype='int64'),
 ('75110 - Paris 10e Arrondissement',
  'BOULEVARD DE MAGENTA'): Int64Index([267, 507, 680, 2172, 2221, 2232, 2296, 2

In [20]:
from folium.plugins import HeatMap

# Création d'une carte centrée sur une localisation avec un niveau de zoom initial de 12
carte1 = fo.Map([48.866667, 2.333333], zoom_start=12)

# Affichage des 25 localisations avec le plus d'accidents en Île-de-France
i = 0
list1 = []

for key in dict_trie_accidents_2022idf.keys():
    # Récupération des coordonnées et du poids pour chaque localisation
    longitude = accidents_2022idf_carac.iloc[dict_trie_accidents_2022idf[key][0]]['Longitude']
    latitude = accidents_2022idf_carac.iloc[dict_trie_accidents_2022idf[key][0]]['Latitude']
    weight = 0.7 * len(dict_trie_accidents_2022idf[key]) / len(dict_trie_accidents_2022idf[('93066 - Saint-Denis', 'AUTOROUTE A1')])

    # Ajout des données dans la liste pour HeatMap
    list1.append([longitude, latitude, weight])
    i = i + 1
    if i > 25:
        break

# Création d'une HeatMap à partir de la liste de données et ajout à la carte
HeatMap(list1).add_to(carte1)

# Sauvegarde de la carte au format HTML
carte1.save("accidents_idf_2022_chaleur.html")
